In [348]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import time

In [349]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [350]:
# Load the CSV file into a DataFrame
df = pd.read_csv('preprocessed_data.csv')
df.head()

,duration,protocol_type,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,...,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,target,Attack Type
0,0,1,0,181,5450,0,0,0,0,0,...,1.0,0.0,0.0,9,9,0.0,0.11,0.0,normal.,normal
1,0,1,0,239,486,0,0,0,0,0,...,1.0,0.0,0.0,19,19,0.0,0.05,0.0,normal.,normal
2,0,1,0,235,1337,0,0,0,0,0,...,1.0,0.0,0.0,29,29,0.0,0.03,0.0,normal.,normal
3,0,1,0,219,1337,0,0,0,0,0,...,1.0,0.0,0.0,39,39,0.0,0.03,0.0,normal.,normal
4,0,1,0,217,2032,0,0,0,0,0,...,1.0,0.0,0.0,49,49,0.0,0.02,0.0,normal.,normal


In [351]:
df['Attack Type'].unique()

array(['normal', 'u2r', 'dos', 'r2l', 'probe'], dtype=object)

In [352]:
corr_matrix = df.iloc[:, :-2].corr()

# Filter the correlation matrix to show only values greater than 0.8
high_corr = corr_matrix[corr_matrix > 0.8]

# Remove self-correlation (correlation of a column with itself) by setting diagonal values to NaN
for i in range(len(high_corr)):
    high_corr.iat[i, i] = None

# Drop rows and columns where there are no values > 0.8
high_corr_filtered = high_corr.dropna(how='all').dropna(axis=1, how='all')

# Display the columns with correlation > 0.8
high_corr_filtered

,hot,is_guest_login,count,srv_count,same_srv_rate,dst_host_srv_count,dst_host_same_src_port_rate
hot,NaN,0.843572,NaN,NaN,NaN,NaN,NaN
is_guest_login,0.843572,NaN,NaN,NaN,NaN,NaN,NaN
count,NaN,NaN,NaN,0.943667,NaN,NaN,0.860579
srv_count,NaN,NaN,0.943667,NaN,NaN,NaN,0.944926
same_srv_rate,NaN,NaN,NaN,NaN,NaN,0.898955,NaN
dst_host_srv_count,NaN,NaN,NaN,NaN,0.898955,NaN,NaN
dst_host_same_src_port_rate,NaN,NaN,0.860579,0.944926,NaN,NaN,NaN


In [353]:
df.drop(['is_guest_login','srv_count','dst_host_srv_count'],axis=1,inplace=True)
df.reset_index(drop=True,inplace=True)

In [354]:
df['Attack Type'].value_counts()

Attack Type
dos       391458
normal     97278
probe       4107
r2l         1126
u2r           52
Name: count, dtype: int64

In [355]:
df.isnull().sum().sum()

0

In [356]:
 #Checking for data imbalance
from sklearn.utils import resample

# Separate majority and minority classes
data_majority = df[df['Attack Type'] == 'dos']
data_minority_5 = df[df['Attack Type'] == 'normal']
data_minority_2 = df[df['Attack Type'] == 'probe']
data_minority_3 = df[df['Attack Type'] == 'r2l']
data_minority_4 = df[df['Attack Type'] == 'u2r']

data_minority_5_upsampled = resample(data_minority_5,replace=True,n_samples=len(data_majority),random_state=42)
data_minority_2_upsampled = resample(data_minority_2,replace=True,n_samples=len(data_majority), random_state=42)
data_minority_3_upsampled = resample(data_minority_3,replace=True,n_samples=len(data_majority), random_state=42)
data_minority_4_upsampled = resample(data_minority_4,replace=True,n_samples=len(data_majority), random_state=42)


data_balanced = pd.concat([data_majority,data_minority_5_upsampled,data_minority_2_upsampled,data_minority_3_upsampled,data_minority_4_upsampled])
#print(data_balanced)

print(data_balanced['Attack Type'].value_counts())

Attack Type
dos       391458
normal    391458
probe     391458
r2l       391458
u2r       391458
Name: count, dtype: int64


In [357]:
df['Attack Type'].isnull().sum()

0

In [358]:
df = df.drop(['target'], axis=1)
print(df.shape)

# Target variable and train set
y = df['Attack Type']  # Make sure this column exists
label = LabelEncoder()
df['Attack Type']= label.fit_transform(y)  # Encode 'Attack Type'

# Check the encoded labels
print("Encoded labels:", df['Attack Type'])

X = df.drop(['Attack Type'], axis=1)  # Drop 'Attack Type' from features

# Scale features
sc = MinMaxScaler()
X = sc.fit_transform(X)

# Split test and train data 
X_train, X_test, y_train, y_test = train_test_split(X,df['Attack Type'] , test_size=0.33, random_state=42)
print("Training data shape:", X_train.shape, y_train.shape)
print("Test data shape:", X_test.shape, y_test.shape)


(494021, 30)
Encoded labels: 0         1
1         1
2         1
3         1
4         1
         ..
494016    1
494017    1
494018    1
494019    1
494020    1
Name: Attack Type, Length: 494021, dtype: int32
Training data shape: (330994, 29) (330994,)
Test data shape: (163027, 29) (163027,)


In [359]:
df['Attack Type'].unique()

array([1, 4, 0, 3, 2])

In [360]:
# Get all mappings
attack_mapping = {index: name for index, name in enumerate(label.classes_)}
print("Mapping of encoded values to original values:", attack_mapping)

Mapping of encoded values to original values: {0: 'dos', 1: 'normal', 2: 'probe', 3: 'r2l', 4: 'u2r'}


In [361]:
df

,duration,protocol_type,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,...,serror_rate,rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,Attack Type
0,0,1,0,181,5450,0,0,0,0,0,...,0.00,0.0,1.0,0.0,0.00,9,0.0,0.11,0.00,1
1,0,1,0,239,486,0,0,0,0,0,...,0.00,0.0,1.0,0.0,0.00,19,0.0,0.05,0.00,1
2,0,1,0,235,1337,0,0,0,0,0,...,0.00,0.0,1.0,0.0,0.00,29,0.0,0.03,0.00,1
3,0,1,0,219,1337,0,0,0,0,0,...,0.00,0.0,1.0,0.0,0.00,39,0.0,0.03,0.00,1
4,0,1,0,217,2032,0,0,0,0,0,...,0.00,0.0,1.0,0.0,0.00,49,0.0,0.02,0.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494016,0,1,0,310,1881,0,0,0,0,0,...,0.00,0.0,1.0,0.0,0.40,86,0.0,0.01,0.05,1
494017,0,1,0,282,2286,0,0,0,0,0,...,0.00,0.0,1.0,0.0,0.00,6,0.0,0.17,0.05,1
494018,0,1,0,203,1200,0,0,0,0,0,...,0.17,0.0,1.0,0.0,0.17,16,0.0,0.06,0.05,1
494019,0,1,0,291,1200,0,0,0,0,0,...,0.00,0.0,1.0,0.0,0.17,26,0.0,0.04,0.05,1


In [362]:
def calculate_tp_tn_fp_fn(confusion_matrix):
    """
    Function to calculate True Positives (TP), True Negatives (TN),
    False Positives (FP), and False Negatives (FN) from a given confusion matrix.

    Args:
    confusion_matrix (numpy.ndarray): The confusion matrix (multiclass)

    Returns:
    dict: A dictionary containing TP, TN, FP, and FN values
    """
    # True Negatives (TN): Normal correctly classified as Normal (Class 0 classified as Class 0)
    TN = confusion_matrix[0, 0]

    # False Positives (FP): Normal misclassified as any Attack (Class 0 misclassified as Class 1-4)
    FP = confusion_matrix[0, 1:].sum()

    # True Positives (TP): Attack correctly classified as Attack (Class 1-4 classified as Class 1-4)
    TP = confusion_matrix[1:, 1:].sum()

    # False Negatives (FN): Attack misclassified as Normal (Class 1-4 misclassified as Class 0)
    FN = confusion_matrix[1:, 0].sum()

    return {
        'True Positives (TP)': TP,
        'True Negatives (TN)': TN,
        'False Positives (FP)': FP,
        'False Negatives (FN)': FN
    }

In [363]:
# Gaussian Naive Bayes
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score, recall_score, f1_score

In [364]:
clfb = GaussianNB()
start_time_train= time.time()

In [365]:
clfb.fit(X_train, y_train)
end_time_train= time.time()

In [366]:
print(f"Training time: {end_time_train - start_time_train:.4f} seconds")

Training time: 0.3593 seconds


In [367]:
start_time_pred_train= time.time()
y_train_pred=clfb.predict(X_train)
end_time_pred_train= time.time()

In [368]:
start_time_pred_test = time.time()
y_test_pred = clfb.predict(X_test)
end_time_pred_test = time.time()

In [369]:
print(f"Prediction time on test data: {end_time_pred_test - start_time_pred_test:.4f} seconds")


Prediction time on test data: 0.5862 seconds


In [370]:
print("Train score is:", clfb.score(X_train, y_train))
print("Test score is:",clfb.score(X_test,y_test))

Train score is: 0.8119724224608301
Test score is: 0.8102461555447871


In [371]:


# Calculate metrics
precision_gaus = precision_score(y_test, y_test_pred, average='weighted')
accuracy_gaus= accuracy_score(y_test, y_test_pred)
recall_gaus= recall_score(y_test, y_test_pred, average='weighted')
f1_gaus= f1_score(y_test, y_test_pred, average='weighted')
cm_gaus= confusion_matrix(y_test, y_test_pred)
r=calculate_tp_tn_fp_fn(cm_gaus)

# Display results
print(f"Precision: {precision_gaus}")
print(f"Accuracy: {accuracy_gaus}")
print(f"Recall: {recall_gaus}")
print(f"F1 Score: {f1_gaus}")
print(f"Confusion Matrix:\n{cm_gaus}")
print(r)

Precision: 0.9675072108558507
Accuracy: 0.8102461555447871
Recall: 0.8102461555447871
F1 Score: 0.8486298746128261
Confusion Matrix:
[[121304    216   7080     12    494]
 [  2905   9301   6089    328  13544]
 [    16      2   1326      1      3]
 [     0      3      4    147    233]
 [     0      0      2      3     14]]
{'True Positives (TP)': 31000, 'True Negatives (TN)': 121304, 'False Positives (FP)': 7802, 'False Negatives (FN)': 2921}


In [372]:
# Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier

# Initialize the model
clfd = DecisionTreeClassifier(criterion="entropy", max_depth=4)

# Train the model
start_time = time.time()
clfd.fit(X_train, y_train)
end_time = time.time()
print("Training time: ", end_time - start_time)

# Predict using the training data
y_train_pred = clfd.predict(X_train)

# Predict using the test data
y_test_pred = clfd.predict(X_test)

#Calculate and display metrics for training and testing
train_accuracy = clfd.score(X_train, y_train)
test_accuracy = clfd.score(X_test, y_test)
print("Train score is:", train_accuracy)
print("Test score is:", test_accuracy)

# Calculate metrics
precision_dec = precision_score(y_test, y_test_pred, average='weighted')
accuracy_dec = accuracy_score(y_test, y_test_pred)
recall_dec = recall_score(y_test, y_test_pred, average='weighted')
f1_dec = f1_score(y_test, y_test_pred, average='weighted')
cm_dec = confusion_matrix(y_test, y_test_pred)
r = calculate_tp_tn_fp_fn(cm_dec)

# Display results
print(f"Precision: {precision_dec}")
print(f"Accuracy: {accuracy_dec}")
print(f"Recall: {recall_dec}")
print(f"F1 Score: {f1_dec}")
print(f"Confusion Matrix:\n{cm_dec}")
print(r)


Training time:  0.7661995887756348
Train score is: 0.9905829108684749
Test score is: 0.9905230421954646
Precision: 0.990204393538205
Accuracy: 0.9905230421954646
Recall: 0.9905230421954646
F1 Score: 0.9900482552140616
Confusion Matrix:
[[128775    261     70      0      0]
 [     5  31510    652      0      0]
 [     8    143   1197      0      0]
 [     0    157    230      0      0]
 [     0     18      1      0      0]]
{'True Positives (TP)': 33908, 'True Negatives (TN)': 128775, 'False Positives (FP)': 331, 'False Negatives (FN)': 13}


C:\Users\sreev\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [373]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494021 entries, 0 to 494020
Data columns (total 30 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   duration                     494021 non-null  int64  
 1   protocol_type                494021 non-null  int64  
 2   flag                         494021 non-null  int64  
 3   src_bytes                    494021 non-null  int64  
 4   dst_bytes                    494021 non-null  int64  
 5   land                         494021 non-null  int64  
 6   wrong_fragment               494021 non-null  int64  
 7   urgent                       494021 non-null  int64  
 8   hot                          494021 non-null  int64  
 9   num_failed_logins            494021 non-null  int64  
 10  logged_in                    494021 non-null  int64  
 11  num_compromised              494021 non-null  int64  
 12  root_shell                   494021 non-null  int64  
 13 

In [374]:
#RandomForest
from sklearn.ensemble import RandomForestClassifier

In [375]:
clfr = RandomForestClassifier(n_estimators=100,max_depth=10,random_state=42)

In [376]:
start_time_train = time.time()
clfr.fit(X_train, y_train)
end_time_train = time.time()
print(f"Training time: {end_time_train - start_time_train:.4f} seconds")

Training time: 23.2506 seconds


In [377]:
start_time_pred_train = time.time()
y_train_pred = clfr.predict(X_train)
end_time_pred_train = time.time()
print(f"Prediction time on training data: {end_time_pred_train - start_time_pred_train:.4f} seconds")

Prediction time on training data: 2.6559 seconds


In [378]:
start_time_pred_test = time.time()
y_test_pred = clfr.predict(X_test)
end_time_pred_test = time.time()
print(f"Prediction time on test data: {end_time_pred_test - start_time_pred_test:.4f} seconds")

Prediction time on test data: 1.1554 seconds


In [379]:
# Train and Test scores
print("Train score is:", clfr.score(X_train, y_train))
print("Test score is:", clfr.score(X_test, y_test))

Train score is: 0.9995679679994199
Test score is: 0.9994050065326603


In [380]:
# Calculate metrics
precision_rand = precision_score(y_test, y_test_pred, average='weighted')
accuracy_rand = accuracy_score(y_test, y_test_pred)
recall_rand = recall_score(y_test, y_test_pred, average='weighted')
f1_rand = f1_score(y_test, y_test_pred, average='weighted')
cm_rand = confusion_matrix(y_test, y_test_pred)
r = calculate_tp_tn_fp_fn(cm_rand)

# Display results
print(f"Precision: {precision_rand:.4f}")
print(f"Accuracy: {accuracy_rand:.4f}")
print(f"Recall: {recall_rand:.4f}")
print(f"F1 Score: {f1_rand:.4f}")
print(f"Confusion Matrix:\n{cm_rand}")
print(r)

Precision: 0.9994
Accuracy: 0.9994
Recall: 0.9994
F1 Score: 0.9994
Confusion Matrix:
[[129102      4      0      0      0]
 [     1  32163      0      3      0]
 [     1     36   1311      0      0]
 [     0     41      0    346      0]
 [     0     10      0      1      8]]
{'True Positives (TP)': 33919, 'True Negatives (TN)': 129102, 'False Positives (FP)': 4, 'False Negatives (FN)': 2}


In [381]:
pip install xgboost



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\sreev\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [382]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

# Initialize XGBoost Classifier (no need for use_label_encoder, it's deprecated)
clfg = xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=6, eval_metric='mlogloss', random_state=42)

# Encode target labels (y_train) from string to numeric
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test) 

In [383]:
import time

# Record the start time for training
start_time = time.time()
# Train the model
clfg.fit(X_train, y_train_encoded)
end_time = time.time()
print(f"Training time: {end_time - start_time:.4f} seconds")


Training time: 7.7082 seconds


In [384]:
# Time the prediction process
start_time = time.time()
y_test_pred_xgb_encoded = clfg.predict(X_test)
end_time = time.time()
print(f"Testing time: {end_time - start_time:.4f} seconds")

Testing time: 0.2758 seconds


In [385]:
y_test_pred_xgb = label_encoder.inverse_transform(y_test_pred_xgb_encoded)

In [386]:
# Calculate performance metrics
precision_xgb = precision_score(y_test, y_test_pred_xgb, average='weighted')
accuracy_xgb = accuracy_score(y_test, y_test_pred_xgb)
recall_xgb = recall_score(y_test, y_test_pred_xgb, average='weighted')
f1_xgb = f1_score(y_test, y_test_pred_xgb, average='weighted')
cm_xgb = confusion_matrix(y_test, y_test_pred_xgb)

# Assuming calculate_tp_tn_fp_fn function is defined to take confusion matrix as input
r = calculate_tp_tn_fp_fn(cm_xgb)

# Display results
print(f"Precision (XGBoost): {precision_xgb:.4f}")
print(f"Accuracy (XGBoost): {accuracy_xgb:.4f}")
print(f"Recall (XGBoost): {recall_xgb:.4f}")
print(f"F1 Score (XGBoost): {f1_xgb:.4f}")
print(f"Confusion Matrix (XGBoost):\n{cm_xgb}")
print(r)

Precision (XGBoost): 0.9998
Accuracy (XGBoost): 0.9998
Recall (XGBoost): 0.9998
F1 Score (XGBoost): 0.9998
Confusion Matrix (XGBoost):
[[129106      0      0      0      0]
 [     3  32159      5      0      0]
 [     0      8   1340      0      0]
 [     0     13      0    374      0]
 [     0      3      1      2     13]]
{'True Positives (TP)': 33918, 'True Negatives (TN)': 129106, 'False Positives (FP)': 0, 'False Negatives (FN)': 3}


In [387]:
from tabulate import tabulate

# Data for all the models with precomputed values
data = [
    ["Random Forest",accuracy_rand,precision_rand,recall_rand,f1_rand,cm_rand],
    ["Decision Tree",accuracy_dec,precision_rand,recall_dec,f1_dec,cm_dec],
    ["Naive Bayes",accuracy_gaus,precision_gaus,recall_gaus,f1_gaus,cm_gaus],
    ["Gradient Boosting",precision_xgb,accuracy_xgb,recall_xgb,f1_xgb,cm_xgb]
]

# Define column names
col_names = ["Model", "Accuracy", "Precision", "Recall", "F1 Score","Confusion Matrix"]

# Print the table
print(tabulate(data, headers=col_names, tablefmt="fancy_grid"))


╒═══════════════════╤════════════╤═════════════╤══════════╤════════════╤════════════════════════════════════════╕
│ Model             │   Accuracy │   Precision │   Recall │   F1 Score │ Confusion Matrix                       │
╞═══════════════════╪════════════╪═════════════╪══════════╪════════════╪════════════════════════════════════════╡
│ Random Forest     │   0.999405 │    0.999404 │ 0.999405 │   0.999383 │ [[129102      4      0      0      0]  │
│                   │            │             │          │            │  [     1  32163      0      3      0]  │
│                   │            │             │          │            │  [     1     36   1311      0      0]  │
│                   │            │             │          │            │  [     0     41      0    346      0]  │
│                   │            │             │          │            │  [     0     10      0      1      8]] │
├───────────────────┼────────────┼─────────────┼──────────┼────────────┼────────────────

In [388]:
print(type(clfr), clfr)
print(type(clfg), clfg)
print(type(clfd), clfd)
print(type(clfb), clfb)


<class 'sklearn.ensemble._forest.RandomForestClassifier'> RandomForestClassifier(max_depth=10, random_state=42)
<class 'xgboost.sklearn.XGBClassifier'> XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)
<class 'sklearn.tree._classes.DecisionTreeClassifier'> DecisionTreeClassifier(cr

In [389]:
import joblib

try:
    # Save the trained models
    joblib.dump(clfr, 'random_forest_model.pkl')
    joblib.dump(clfg, 'gradient_boosting_model.pkl')
    joblib.dump(clfd, 'decision_tree_model.pkl')
    joblib.dump(clfb, 'naive_bayes_model.pkl')
    print("Models saved successfully!")
except Exception as e:
    print(f"Error saving models: {e}")


Models saved successfully!
